In [113]:
# imports

import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from agents.deals import Deal, ScrapedDeal, DealSelection
import agents.scanner_agent

In [114]:
import importlib
import agents.deals
importlib.reload(agents.deals)
from agents.deals import Deal,ScrapedDeal, DealSelection

In [115]:
import importlib
import agents.scanner_agent
importlib.reload(agents.scanner_agent)

<module 'agents.scanner_agent' from 'C:\\Users\\rayen\\LLM Engineering\\pricer_agent\\agents\\scanner_agent.py'>

In [116]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gemini-2.5-flash'
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [117]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:50<00:00, 22.08s/it]


In [118]:
len(deals)

50

In [119]:
deals[1].describe()

'Title: Best of Tech Deals at eBay: Up to 89% off + free shipping\nDetails: There are lots of great discounts on new and refurbished smartphones, smart watches, headphones, soundbars, and lots more in this tech outlet at eBay. Top brands like Apple, Samsung, Sony, and Bose are included. Shop Now at eBay\nFeatures: \nURL: https://www.dealnews.com/Best-of-Tech-Deals-at-eBay-Up-to-89-off-free-shipping/21751562.html?iref=rss-c142'

In [120]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [121]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [122]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Apple Deals at Costco: Up to $350 off + shipping varies
Details: Shop up to $350 off nearly 50 products from Apple. You will find deals on AirPods, iPads, Apple Watches, MacBooks, and more. Shipping varies by item. Shop Now at Costco
Features: 
URL: https://www.dealnews.com/Apple-Deals-at-Costco-Up-to-350-off-shipping-varies/21751567.html?iref=rss-c142

Title: Best 

In [94]:
def get_recommendations():
    try:
        model = genai.GenerativeModel(MODEL)
        response = model.generate_content(user_prompt)
        result = response.text
    except Exception as e :
        self.log(f"Gemini API Error: {e}")
        
    return result

In [95]:
def parse_price(price_str: str) -> float:
    """
    Parse price string like "$210" to float
    """
    if isinstance(price_str, str):
        # Remove $ and any commas, convert to float
        return float(price_str.replace('$', '').replace(',', ''))
    return float(price_str)

def extract_json(text: str) -> str:
    """
    Extract JSON from text that might contain markdown code blocks
    """
    # Remove markdown code blocks
    text = re.sub(r'```json\n', '', text)
    text = re.sub(r'```\n?', '', text)
    text = text.strip()
    
    # Try to find JSON content - check for both objects {} and arrays []
    object_start = text.find('{')
    array_start = text.find('[')
    
    # Determine which comes first (or if only one exists)
    if object_start != -1 and (array_start == -1 or object_start < array_start):
        # JSON object
        start = object_start
        end = text.rfind('}') + 1
    elif array_start != -1:
        # JSON array
        start = array_start
        end = text.rfind(']') + 1
    else:
        # No JSON found, return as is
        return text
    
    if start != -1 and end != 0:
        return text[start:end]
    return text

def get_recommendations():
    try:
        # This would be your actual Gemini API call
        model = genai.GenerativeModel(MODEL)
        response = model.generate_content(user_prompt)
        reply = response.text
        
        
        print("RAW Gemini reply:\n", repr(reply))  # Debug: show raw text
        clean_text = extract_json(reply)
        print("Cleaned JSON text:\n", clean_text)  # Debug: show stripped version
        parsed = json.loads(clean_text)
        print("Parsed JSON type:", type(parsed))  # Debug: show type
        
        # Determine deal list location
        deals_data = None
        if isinstance(parsed, dict):
            print("Parsed keys:", parsed.keys())  # Extra debug
            for key in ["selected_deals", "deals", "promising_deals"]:
                deals_data = parsed.get(key)
                if deals_data:
                    break
        elif isinstance(parsed, list):
            deals_data = parsed
        else:
            raise ValueError("Parsed JSON is not a list or dict")
        
        if not deals_data:
            raise ValueError("No deals found in parsed JSON")
        
        deals = [
            Deal(
                title=deal["title"],
                product_description=deal["product_description"],
                price=parse_price(deal["price"]),
                url=deal.get("url")  # Use .get() since URL might not exist
            )
            for deal in deals_data
        ]
        
        return DealSelection(deals=deals)
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")
    return None

In [96]:
result = get_recommendations()

RAW Gemini reply:
 '```json\n[\n  {\n    "title": "Refurb Unlocked Samsung Galaxy S21 256GB 5G Android Smartphone",\n    "price": "$210",\n    "product_description": "This is a refurbished Samsung Galaxy S21, an unlocked 5G Android smartphone featuring 256GB of internal storage. It runs on Android 11 and is powered by a Qualcomm SM8350 Snapdragon 888 Octa-Core Processor. The device boasts a vibrant 6.2-inch 2400x1080 120Hz touchscreen, capable of 8K video recording. For photography, it includes a versatile camera system with a 64MP telephoto lens, 12MP wide and ultra-wide cameras, and a 10MP front camera, all within model SM-G991UZAEXAA."\n  },\n  {\n    "title": "HP Victus Ryzen 7 15.6\\" Laptop w/ NVIDIA GeForce RTX 4050",\n    "price": "$650",\n    "product_description": "The HP Victus is a high-performance 15.6-inch gaming laptop, featuring an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU and NVIDIA GeForce RTX 4050 graphics with 6GB of dedicated memory, making it suitable for demanding gam

In [97]:
print(result)

deals=[Deal(product_description='This is a refurbished Samsung Galaxy S21, an unlocked 5G Android smartphone featuring 256GB of internal storage. It runs on Android 11 and is powered by a Qualcomm SM8350 Snapdragon 888 Octa-Core Processor. The device boasts a vibrant 6.2-inch 2400x1080 120Hz touchscreen, capable of 8K video recording. For photography, it includes a versatile camera system with a 64MP telephoto lens, 12MP wide and ultra-wide cameras, and a 10MP front camera, all within model SM-G991UZAEXAA.', price=210.0, url=None), Deal(product_description='The HP Victus is a high-performance 15.6-inch gaming laptop, featuring an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU and NVIDIA GeForce RTX 4050 graphics with 6GB of dedicated memory, making it suitable for demanding games and creative tasks. It comes equipped with 16GB of RAM and a 512GB SSD for fast boot times and ample storage. The visual experience is delivered through a 15.6-inch 1920x1080 (1080p) IPS display, and the system runs on 

In [98]:
len(result.deals)

5

In [99]:
result.deals[3]

Deal(product_description='This Vizio 50-inch MQX-Series Smart TV delivers stunning visuals with its 4K resolution and a smooth 120Hz refresh rate, ideal for immersive viewing and gaming. It supports HDR10 and Dolby Vision, enhancing colors, contrast, and brightness for a more lifelike picture. As a smart TV, it offers easy access to streaming content, and its specific model is M50QXM-K01.', price=398.0, url=None)

In [123]:
from agents.scanner_agent import ScannerAgent

In [124]:
agent = ScannerAgent()
result = agent.scan_gemini()

RAW Gemini reply:
 '```json\n{\n    "deals": [\n        {\n            "product_description": "This HP Victus laptop is a robust machine designed for demanding applications and gaming, powered by an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU. It comes equipped with 16GB of RAM and a fast 512GB SSD, ensuring quick load times and efficient multitasking. The visual experience is delivered through a 15.6-inch 1920x1080 (1080p) IPS display, offering crisp images and wide viewing angles. For graphics-intensive tasks and modern games, it features a dedicated NVIDIA GeForce RTX 4050 6GB graphics card. The laptop runs on Windows 11 Home, providing a contemporary and user-friendly operating system.",\n            "price": 650.00,\n            "url": "https://www.dealnews.com/products/HP/HP-Victus-Ryzen-7-15-6-Laptop-w-NVIDIA-Ge-Force-RTX-4050/491498.html?iref=rss-c39"\n        },\n        {\n            "product_description": "The Dell Ultra 7 265 Slim Desktop is a compact yet powerful computing solut

In [125]:
print(result)

deals=[Deal(product_description='This HP Victus laptop is a robust machine designed for demanding applications and gaming, powered by an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU. It comes equipped with 16GB of RAM and a fast 512GB SSD, ensuring quick load times and efficient multitasking. The visual experience is delivered through a 15.6-inch 1920x1080 (1080p) IPS display, offering crisp images and wide viewing angles. For graphics-intensive tasks and modern games, it features a dedicated NVIDIA GeForce RTX 4050 6GB graphics card. The laptop runs on Windows 11 Home, providing a contemporary and user-friendly operating system.', price=650.0, url='https://www.dealnews.com/products/HP/HP-Victus-Ryzen-7-15-6-Laptop-w-NVIDIA-Ge-Force-RTX-4050/491498.html?iref=rss-c39'), Deal(product_description='The Dell Ultra 7 265 Slim Desktop is a compact yet powerful computing solution, featuring an Intel Core Ultra 7 265 20-Core processor that boasts a clock speed range of 1.8GHz to 5.3GHz, suitable for va